# Import file

In [1]:
import pandas as pd
data_path = '/home/antoine.laborde/Documents/data/ppty-identification/goldset_dedup_grouped.csv'
gold_set = pd.read_csv(data_path)
data_path = '/home/antoine.laborde/Documents/data/ppty-identification/mapping_id_tag.xlsx'
tag_id_mapping = pd.read_excel(data_path)

In [2]:
gold_set

,idannonce,cp,ville,codeinsee,descriptif_fr,px,surface,annee_construct,nb_pieces,latitude,...,districts_name,region_id,min_px,max_px,min_surface,max_surface,min_nbpiece,max_nbpiece,groups,len_group
0,152917265,38100,Grenoble,380185.0,grenoble allee de la pelouse en bordure du par...,88000.0,80.08,NaN,4.0,45.162621,...,Arlequin,252,83600.0,92400.0,76.076,84.084,3.0,5.0,"[154661865, 151927233, 151927251, 131612729, 1...",9
1,151210099,38100,Grenoble,380185.0,era immobilier gresivaudan a trouve pour vous ...,158970.0,70.00,1960.0,3.0,45.180660,...,Allies-Clos d'Or,252,151021.5,166918.5,66.500,73.500,2.0,4.0,"[154138775, 150604827, 152178193, 138237015, 1...",14
2,150907823,38130,Échirolles,380151.0,iad france sophie armand vous propose echiroll...,125000.0,63.00,1970.0,4.0,45.147226,...,Comboire,252,118750.0,131250.0,59.850,66.150,3.0,5.0,"[147619281, 153267117, 151856983, 151785547, 1...",16
3,148550245,38000,Grenoble,380185.0,appartement residence etudiante marie curie de...,67000.0,18.00,NaN,1.0,45.194349,...,Berriat-Ampere,252,63650.0,70350.0,17.100,18.900,0.0,2.0,[],0
4,152658935,38400,Saint Martin d Heres,380421.0,secteur parc jo blanchon au calme et proche tr...,199000.0,99.00,NaN,6.0,45.173628,...,Renaudie-La Plaine,252,189050.0,208950.0,94.050,103.950,5.0,7.0,[151928147],1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3638,150900765,75010,Paris 10ème,750110.0,hauteville bonne nouvelle au sein d un immeubl...,630000.0,51.00,NaN,3.0,NaN,...,NaN,239,598500.0,661500.0,48.450,53.550,2.0,4.0,"[152658239, 154295907, 151115771]",3
3639,148296771,93300,Aubervilliers,930001.0,a min de metro porte la villette immeuble secu...,265000.0,74.00,NaN,4.0,NaN,...,NaN,239,251750.0,278250.0,70.300,77.700,3.0,5.0,"[152471347, 154517705, 152241143, 152306431, 1...",7
3640,152968579,75017,Paris 17ème,750117.0,marche lebon grand pieces renove de m dans imm...,667800.0,53.00,1900.0,2.0,NaN,...,NaN,239,634410.0,701190.0,50.350,55.650,1.0,3.0,"[152557609, 154419047, 153712073, 154670377, 1...",8
3641,145561525,93400,St Ouen,930070.0,laforet saint ouen dans le quartier recherche ...,525000.0,86.80,NaN,4.0,NaN,...,NaN,239,498750.0,551250.0,82.460,91.140,3.0,5.0,"[145549993, 153617397, 154195785, 154032987]",4


In [25]:
import requests
import numpy as np
import copy
def tag_to_cdn(photo_tag, width = 300):
    if isinstance(photo_tag, str):
        url = f"https://v.seloger.com/s/width/{width}/visuels/{'/'.join(photo_tag[:4])}/{photo_tag}.jpg"
        return url
        
def id_to_tag(ads_id, mapping):
    return mapping[mapping.idannonce==ads_id]['tagphoto'].values.tolist()

def id_to_cdn(ads_id, mapping):
    cdn_list = []
    for tag in id_to_tag(ads_id, mapping):
        cdn_list.append(tag_to_cdn(tag))
    return cdn_list

def check_jsonl(line):
    # Get list of image-url
    if len(line['images'])==0:
        return False
    option_list = copy.deepcopy(line['options'])
    for opt in line['options']:
        if len(opt['images'])==0:
            option_list.remove(opt)
    if len(option_list)==0:
        return False
    else:
        line['options'] = option_list
        return line
    
            
        

In [26]:
json_line_list = []
for ind, row in gold_set.iterrows():
    
    line = {
        "id": str(row.idannonce),
        "text": row.descriptif_fr,
        "images": id_to_cdn(row.idannonce, tag_id_mapping)
    }
    # Remove
    
    options_list = []
    for id_linked in eval(row.groups):
        text_descriptif = gold_set[gold_set.idannonce==id_linked].descriptif_fr.values[0]
        options_dict = {
            "id": id_linked,
            "text": text_descriptif,
            "images": id_to_cdn(id_linked, tag_id_mapping)
        }
        options_list.append(options_dict)
    line["options"] = options_list
    if check_jsonl(line):
        json_line_list.append(line)
    print(f'{ind+1}/{gold_set.shape[0]}')
        

1/3643
2/3643
3/3643
4/3643
5/3643
6/3643
7/3643
8/3643
9/3643
10/3643
11/3643
12/3643
13/3643
14/3643
15/3643
16/3643
17/3643
18/3643
19/3643
20/3643
21/3643
22/3643
23/3643
24/3643
25/3643
26/3643
27/3643
28/3643
29/3643
30/3643
31/3643
32/3643
33/3643
34/3643
35/3643
36/3643
37/3643
38/3643
39/3643
40/3643
41/3643
42/3643
43/3643
44/3643
45/3643
46/3643
47/3643
48/3643
49/3643
50/3643
51/3643
52/3643
53/3643
54/3643
55/3643
56/3643
57/3643
58/3643
59/3643
60/3643
61/3643
62/3643
63/3643
64/3643
65/3643
66/3643
67/3643
68/3643
69/3643
70/3643
71/3643
72/3643
73/3643
74/3643
75/3643
76/3643
77/3643
78/3643
79/3643
80/3643
81/3643
82/3643
83/3643
84/3643
85/3643
86/3643
87/3643
88/3643
89/3643
90/3643
91/3643
92/3643
93/3643
94/3643
95/3643
96/3643
97/3643
98/3643
99/3643
100/3643
101/3643
102/3643
103/3643
104/3643
105/3643
106/3643
107/3643
108/3643
109/3643
110/3643
111/3643
112/3643
113/3643
114/3643
115/3643
116/3643
117/3643
118/3643
119/3643
120/3643
121/3643
122/3643
123/3643
1

932/3643
933/3643
934/3643
935/3643
936/3643
937/3643
938/3643
939/3643
940/3643
941/3643
942/3643
943/3643
944/3643
945/3643
946/3643
947/3643
948/3643
949/3643
950/3643
951/3643
952/3643
953/3643
954/3643
955/3643
956/3643
957/3643
958/3643
959/3643
960/3643
961/3643
962/3643
963/3643
964/3643
965/3643
966/3643
967/3643
968/3643
969/3643
970/3643
971/3643
972/3643
973/3643
974/3643
975/3643
976/3643
977/3643
978/3643
979/3643
980/3643
981/3643
982/3643
983/3643
984/3643
985/3643
986/3643
987/3643
988/3643
989/3643
990/3643
991/3643
992/3643
993/3643
994/3643
995/3643
996/3643
997/3643
998/3643
999/3643
1000/3643
1001/3643
1002/3643
1003/3643
1004/3643
1005/3643
1006/3643
1007/3643
1008/3643
1009/3643
1010/3643
1011/3643
1012/3643
1013/3643
1014/3643
1015/3643
1016/3643
1017/3643
1018/3643
1019/3643
1020/3643
1021/3643
1022/3643
1023/3643
1024/3643
1025/3643
1026/3643
1027/3643
1028/3643
1029/3643
1030/3643
1031/3643
1032/3643
1033/3643
1034/3643
1035/3643
1036/3643
1037/3643
1038/364

1761/3643
1762/3643
1763/3643
1764/3643
1765/3643
1766/3643
1767/3643
1768/3643
1769/3643
1770/3643
1771/3643
1772/3643
1773/3643
1774/3643
1775/3643
1776/3643
1777/3643
1778/3643
1779/3643
1780/3643
1781/3643
1782/3643
1783/3643
1784/3643
1785/3643
1786/3643
1787/3643
1788/3643
1789/3643
1790/3643
1791/3643
1792/3643
1793/3643
1794/3643
1795/3643
1796/3643
1797/3643
1798/3643
1799/3643
1800/3643
1801/3643
1802/3643
1803/3643
1804/3643
1805/3643
1806/3643
1807/3643
1808/3643
1809/3643
1810/3643
1811/3643
1812/3643
1813/3643
1814/3643
1815/3643
1816/3643
1817/3643
1818/3643
1819/3643
1820/3643
1821/3643
1822/3643
1823/3643
1824/3643
1825/3643
1826/3643
1827/3643
1828/3643
1829/3643
1830/3643
1831/3643
1832/3643
1833/3643
1834/3643
1835/3643
1836/3643
1837/3643
1838/3643
1839/3643
1840/3643
1841/3643
1842/3643
1843/3643
1844/3643
1845/3643
1846/3643
1847/3643
1848/3643
1849/3643
1850/3643
1851/3643
1852/3643
1853/3643
1854/3643
1855/3643
1856/3643
1857/3643
1858/3643
1859/3643
1860/3643


2593/3643
2594/3643
2595/3643
2596/3643
2597/3643
2598/3643
2599/3643
2600/3643
2601/3643
2602/3643
2603/3643
2604/3643
2605/3643
2606/3643
2607/3643
2608/3643
2609/3643
2610/3643
2611/3643
2612/3643
2613/3643
2614/3643
2615/3643
2616/3643
2617/3643
2618/3643
2619/3643
2620/3643
2621/3643
2622/3643
2623/3643
2624/3643
2625/3643
2626/3643
2627/3643
2628/3643
2629/3643
2630/3643
2631/3643
2632/3643
2633/3643
2634/3643
2635/3643
2636/3643
2637/3643
2638/3643
2639/3643
2640/3643
2641/3643
2642/3643
2643/3643
2644/3643
2645/3643
2646/3643
2647/3643
2648/3643
2649/3643
2650/3643
2651/3643
2652/3643
2653/3643
2654/3643
2655/3643
2656/3643
2657/3643
2658/3643
2659/3643
2660/3643
2661/3643
2662/3643
2663/3643
2664/3643
2665/3643
2666/3643
2667/3643
2668/3643
2669/3643
2670/3643
2671/3643
2672/3643
2673/3643
2674/3643
2675/3643
2676/3643
2677/3643
2678/3643
2679/3643
2680/3643
2681/3643
2682/3643
2683/3643
2684/3643
2685/3643
2686/3643
2687/3643
2688/3643
2689/3643
2690/3643
2691/3643
2692/3643


3433/3643
3434/3643
3435/3643
3436/3643
3437/3643
3438/3643
3439/3643
3440/3643
3441/3643
3442/3643
3443/3643
3444/3643
3445/3643
3446/3643
3447/3643
3448/3643
3449/3643
3450/3643
3451/3643
3452/3643
3453/3643
3454/3643
3455/3643
3456/3643
3457/3643
3458/3643
3459/3643
3460/3643
3461/3643
3462/3643
3463/3643
3464/3643
3465/3643
3466/3643
3467/3643
3468/3643
3469/3643
3470/3643
3471/3643
3472/3643
3473/3643
3474/3643
3475/3643
3476/3643
3477/3643
3478/3643
3479/3643
3480/3643
3481/3643
3482/3643
3483/3643
3484/3643
3485/3643
3486/3643
3487/3643
3488/3643
3489/3643
3490/3643
3491/3643
3492/3643
3493/3643
3494/3643
3495/3643
3496/3643
3497/3643
3498/3643
3499/3643
3500/3643
3501/3643
3502/3643
3503/3643
3504/3643
3505/3643
3506/3643
3507/3643
3508/3643
3509/3643
3510/3643
3511/3643
3512/3643
3513/3643
3514/3643
3515/3643
3516/3643
3517/3643
3518/3643
3519/3643
3520/3643
3521/3643
3522/3643
3523/3643
3524/3643
3525/3643
3526/3643
3527/3643
3528/3643
3529/3643
3530/3643
3531/3643
3532/3643


In [30]:
import jsonlines
nbr_element_in_task = 10
with jsonlines.open('./task.jsonl', mode='w') as writer:
    writer.write_all(json_line_list[:nbr_element_in_task])

In [31]:
json_line_list[0]['images']

['https://v.seloger.com/s/width/300/visuels/2/5/4/c/254c1izwwxof7juyehto1gnngy7kins7h7r3xqkb8.jpg',
 'https://v.seloger.com/s/width/300/visuels/0/b/w/x/0bwxsqsn5b7lk7sqq92jywh0lhtnftq1gbnw0y6ok.jpg',
 'https://v.seloger.com/s/width/300/visuels/0/z/z/3/0zz3gn1jvkvxfqzrdfpcs4nfkas6etn775u5a66vo.jpg',
 'https://v.seloger.com/s/width/300/visuels/1/z/x/3/1zx3nvirxibh3l754b5jw4cm48mkv0p2oa2ms56xg.jpg',
 'https://v.seloger.com/s/width/300/visuels/1/m/2/f/1m2fm1julrshbjle58j9bstimjhua71m99gxncvo4.jpg',
 'https://v.seloger.com/s/width/300/visuels/0/u/d/q/0udq9npc9rw8b1kss4mm1s4whb8jjyo95c66llwt0.jpg',
 'https://v.seloger.com/s/width/300/visuels/0/8/5/3/08539uyrdrc4krl6ab1sfigjd1h9aerx4s85upx78.jpg',
 'https://v.seloger.com/s/width/300/visuels/2/0/y/q/20yqyye8z66n0zbf2qqjwg34ms3y31zgibrh9wcq8.jpg',
 'https://v.seloger.com/s/width/300/visuels/0/d/y/2/0dy2u4vbauubl8dvj85ksdveertcufub578s85cgk.jpg']

In [29]:
for opt in json_line_list[0]['options']:
    print(opt['id'])
    print(opt['images'])

154661865
['https://v.seloger.com/s/width/300/visuels/0/e/f/c/0efcfs26dmlvhe63kgkt3707y2hvcm09e69j2akfk.jpg', 'https://v.seloger.com/s/width/300/visuels/1/v/o/7/1vo7794mc02rh9jb1atc6hqw5kwy96jlijtj97w0g.jpg', 'https://v.seloger.com/s/width/300/visuels/2/a/s/g/2asgoqia0g6wvajrunvnlhmckvxvfa3lrnp3k5wj4.jpg', 'https://v.seloger.com/s/width/300/visuels/0/d/c/t/0dctur513f2wn7vf1wbkajx16rbny2rtn11xay380.jpg', 'https://v.seloger.com/s/width/300/visuels/0/8/4/w/084w655g6pzmnx6cpdy9n4mm2zeinwopu39gnb7xs.jpg', 'https://v.seloger.com/s/width/300/visuels/0/h/2/4/0h2425lo0bg6qrbk2vgde7uoxn05d2pmklb1gpt68.jpg', 'https://v.seloger.com/s/width/300/visuels/0/z/5/s/0z5sehnk9mazk1k8chztc75cr8dfdxqnwhiamsn9s.jpg', 'https://v.seloger.com/s/width/300/visuels/0/1/k/k/01kkx09etbf59ewzksgtu2d3ylwfuoyyamd2wwpu8.jpg', 'https://v.seloger.com/s/width/300/visuels/2/8/1/6/28168370l6ao4yqm2gnbkzhhlmewccwvqf6ekbpww.jpg']
151927233
['https://v.seloger.com/s/width/300/visuels/1/k/c/a/1kcavwtj0icclc8ktmjrv3cuh3xsy1tw4mds